# Práctica Métodos de Simulación - Parte 3

## Elena Rivas, Teresa Grau, Ignacio Casso

## Enunciado

**Enfriamiento simulado. El problema de las n reinas.**

El problema de las n-reinas consiste en colocar n reinas en un tablero de ajedrez de
n x n de tal manera que no sea posible que dos reinas se capturen entre si, es decir,
que no estén en la misma fila, ni en la misma columna ni en la misma diagonal. Se
dice que hay una colisión si hay dos reinas que se pueden capturar entre si.

Se trata pues de encontrar una configuración (elegir las n celdas donde colocar a las
reinas) que minimice el número total de colisiones. Obtener la solución óptima para
n=9.

Mostrar gráficamente la evolución del valor de la función fitness y del parámetro
temperatura considerados a lo largo de las iteraciones de la metaheurı́stica.

## Solución

Proponemos dos implementaciones distintas para el algoritmo de enfraimiento simulado, que se diferencian en el espacio de búsqueda y en la codificación de las soluciones. En la primera consideramos todo el espacio de búsqueda, es dicir, todas las posibles configuraciones del tablero con $n$ reinas en él. En la segunda, observamos que por necesidad debe haber una reina en cada fila y otra en cada columna, y codificamos las soluciones como permutaciones de $n$ elementos, reduciendo el espacio de soluciones de un cardinal de $(n^2)^n$ a uno de $n!$. Para cada una de ellas se exploran distintas elecciones de parámetros.

In [111]:
from random import random, randint

## Primera Solución

### Codificación de las soluciones

Codificaremos las soluciones como listas de 9 posiciones, siendo cada posición una tupla de 2 elementos. Para evitar redundancia en la codificación, se impone que esa lista esté ordenada (lexicográficamente), y para evitar soluciones no factibles, se prohiben los elementos repetidos en una lista.

### Función objetivo

La función objetivo a minimizar será el número de colisiones entre reinas.

In [92]:
def num_collisions(positions):
    
    if positions==[]:
        return 0
    else:
        queen, positions2 = positions[0], positions[1:]
        num_colls = 0
        for queen2 in positions2:
            if collision(queen,queen2):
                num_colls += 1
        return num_colls + num_collisions(positions2)
    
def collision(queen1,queen2):
    i1,j1 = queen1
    i2,j2 = queen2
    
    if i1==i2 or j1==j2 or abs(i1-i2)==abs(j1-j2):
        return True
    else:
        return False

In [93]:
def numCollisionsFitness(sol,n):
    return num_collisions(sol)

Otras funcione objetivos posibles podrían ser... **TODO**

### Entorno de un punto en el espacio de bísqueda

Proponemos los siguientes entornos para una solución:

La primera, las soluciones obtenidas al mover una reina una posición en cualquier dirección.

In [139]:
moves = [(0,1),(0,-1),(1,0),(-1,0),(1,1),(1,-1),(-1,1),(-1,-1)]

def oneMoveRandomNeighbor(sol,n):
    queens = sol[:] # copy
    i = randint(0,n-1)
    j = randint(0,7)
    qx,qy = queens[i]
    mx,my = moves[j]
    nqx,nqy = qx+mx, qy+my
    if 0<nqx and 0 < nqy and n >= nqx and n>=nqy:
        new_queen = (nqx,nqy)
    else:
        new_queen = qx,qy
    queens[i] = new_queen
    return queens

# TODO: seleccionar otro movimiento si el elegido te saca del tablero
# TODO: ordenar solución y asegurarse de que es factible (no hay dos reinas en la misma posición)
# Por ahora no lo hacemos, ya que funciona aún así

**TODO:** Otras alternativas

### Vaior inicial de la temperatura

Proponemos los siguientes valores iniciales de temperatura:

In [97]:
def standardInitialTemp():
    return 0.9

### Variación de la temperatura

Proponemos las siguientes actualizaciones de temperatura:

En primer lugar, la propuesta en las transparencias, con $L=10,~ \alpha = 0.95$

In [143]:
def updateTempStandard(T,i):
    if i%10 == 0:
        newT = 0.95*T
    else:
        newT = T
    return newT

### Probabilidad de aceptación de nuevas soluciones

La estándar, entiendo que si se quisiera probar opciones de Serafini (1992), no sólo aparecería como referencia si no que tambien estaría en el moodle

**Duda:** No debería estar la probabilidad entre 0 y 1?

In [146]:
from math import exp

In [147]:
def prob_aceptacion_standard(f1,f,T):
    x = -(f1-f)/T
    return exp(x)

### Criterio de parada

Proponemos los siguientes criterios de parada:

El primero, haber encontrado la solución óptima

In [107]:
def solutionFound(f,_):
    return f==0

**TODO:** Criterios de parada de los apuntes

### Solución inicial

Proponemos varias soluciones iniciales para el algoritmo.

La primera es simplemente todas las reinas en la primera fila:

In [69]:
def initialSolSameRow(n):
    return [(1,i+1) for i in range(n)]

La segunda será una solución factible pseudo-aleatoria:

In [75]:
# TODO:

La tercera será una solución factible buena aunque no óptima, calculada mediante un algoritmo voraz:

In [76]:
# TODO:

In [78]:
initialSols=[initialSolSameRow]

### Algoritmo

In [62]:
from random import random

In [150]:
def queens(n):
    sol = initialSol(n)
    f = fitness(sol,n)
    best_sol = sol
    best_f = f
    T = initialTemp()
    fitness_values=[f]
    temp=[T]
    i=1
    while not criterio_parada(best_f,i):
        new_sol = random_neighbor(sol,n)
        new_f = fitness(new_sol,n)
        if new_f < f:
            sol = new_sol
            f = new_f
            if f < best_f:
                best_sol=sol
                best_f = f
        else:
            u = random()
            p = prob_aceptacion(new_f,f,T)
            if p > u:
                sol = new_sol
                f = new_f
        update_temp(T,i)
        i += 1
    return sol

In [151]:
initialSol=initialSolSameRow
fitness = numCollisionsFitness
initialTemp = standardInitialTemp
criterio_parada = solutionFound
random_neighbor = oneMoveRandomNeighbor
update_temp = updateTempStandard
prob_aceptacion = prob_aceptacion_standard

In [152]:
queens(9)

[(8, 1), (3, 2), (6, 6), (5, 3), (1, 5), (2, 8), (7, 9), (9, 4), (4, 7)]

## Segunda Solución